In [61]:
import math
import pandas as pd
import numpy as np
import random
from deap import creator, base, tools, algorithms
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

def avg(l):
    """
    Returns the average between list elements
    """
    return (sum(l)/float(len(l)))


def calculate_equivalence_classes(data, condition_attrs):
    # 取出条件属性的列索引
    # 设置条件属性为索引
    # print(condition_attrs)
    data_indexed = data.set_index(condition_attrs)
    # 按照索引分组并将分组内结果合并为列表
    equivalence_classes = data_indexed.groupby(level=condition_attrs)[data.columns[0]].apply(list).to_dict()
    return equivalence_classes

def getFitness(data,decision_attr,individual):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    feature_columns = [col for col in data.columns if col not in ['id','target']]
    selected_features = [feature_columns[i] for i in range(len(feature_columns)) if individual[i] == 1]

    equivalence_classes = calculate_equivalence_classes(data,selected_features)
    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            # print(len(x))
            if(len(x)>total_records*0.05):
                total_rule+=len(x)
                y = 1-len(x) / class_size
                rule_confidences.append(y)
            if (len(x)>0):
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)
    if count == 0:
        return(999,);
    # if total_rule/total_records<0.2:
    #     return (999,);

    avg_confidence = np.sum(rule_confidences) / count +0.7*(entropy_D-entropy_DB)
    return(avg_confidence,)

def geneticAlgorithm(df,y,n_population, n_generation):
    """
    Deap global variables
    Initialize variables to use eaSimple
    """
    random.seed(42)
    # create individual
    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", list, fitness=creator.FitnessMin)

    # create toolbox
    toolbox = base.Toolbox()
    toolbox.register("attr_bool", random.randint, 0, 1)
    toolbox.register("individual", tools.initRepeat,
                     creator.Individual, toolbox.attr_bool, len(X.columns))
    toolbox.register("population", tools.initRepeat, list,
                     toolbox.individual)
    toolbox.register("evaluate", getFitness,df,y)
    toolbox.register("mate", tools.cxOnePoint)
    toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)
    toolbox.register("select", tools.selTournament, tournsize=3)

    # initialize parameters
    pop = toolbox.population(n=n_population)
    hof = tools.HallOfFame(n_population * n_generation)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    # stats.register("max", np.max)

    # genetic algorithm
    pop, log = algorithms.eaSimple(pop, toolbox, cxpb=0.5, mutpb=0.2,
                                   ngen=n_generation, stats=stats, halloffame=hof,
                                   verbose=True)

    # return hall of fame
    top_5_hof = sorted(hof.items, key=lambda x: x.fitness.values[0])[:1]
    for i, ind in enumerate(top_5_hof):
        print(f"Top {i+1} solution: {ind}\nFitness: {ind.fitness.values[0]}\n")
    return top_5_hof



In [62]:
from sklearn.neighbors import KNeighborsClassifier

def bestIndividual_KNN(hof,df):
    k = 5
    knn = KNeighborsClassifier(n_neighbors=k)
    for ind in hof:
        df = pd.read_csv('../../data/iono/ionosphere.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y=df["target"]
        X=df[selected_features]
        kf = KFold(n_splits=10, shuffle=True, random_state=42)
        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            knn.fit(X_train, y_train)
            y_pred = knn.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)
        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
        return mean_accuracy,selected_features

In [63]:
from sklearn.model_selection import StratifiedKFold


def bestIndividual_SVM(hof, df):
    def SVM(trainX, trainY, testX, testY):
        scaler = StandardScaler()
        scaler.fit(trainX)
        trainX3 = scaler.transform(trainX)
        testX3 = scaler.transform(testX)
        clf = svm.SVC(C=10, gamma=0.8, max_iter=2000)
        clf.fit(trainX3, trainY)
        test_result = clf.predict(testX3)
        precision = np.sum(test_result == testY) / testY.shape[0]
        return precision

    maxaccuracy = 0.0
    for ind in hof:
        df = pd.read_csv('../../data/iono/ionosphere.csv')
        feature_columns = [col for col in df.columns if col not in ['id', 'target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        Y = df["target"]
        X = df[selected_features]

        # Use stratified k-fold cross-validation
        skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
        scores = []
        for train_index, test_index in skf.split(X, Y):
            trainX, testX = X.iloc[train_index], X.iloc[test_index]
            trainY, testY = Y.iloc[train_index], Y.iloc[test_index]
            score = SVM(trainX, trainY, testX, testY)
            scores.append(score)
        avg_score = np.mean(scores)

        if avg_score > maxaccuracy:
            maxaccuracy = avg_score
            best_feature = selected_features

    return maxaccuracy, best_feature

In [64]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier


def bestIndividual_DT(hof, df):
    for ind in hof:
        df = pd.read_csv('../../data/iono/ionosphere.csv')
        feature_columns = [col for col in df.columns if col not in ['id','target']]
        selected_features = [feature_columns[i] for i in range(len(feature_columns)) if ind[i] == 1]
        X = df[selected_features]
        Y = df['target']
        dt = DecisionTreeClassifier()
        kf = KFold(n_splits=10, shuffle=True, random_state=42)

        accuracy_scores = []
        for train_index, test_index in kf.split(X):
            X_train, X_test = X.iloc[train_index], X.iloc[test_index]
            y_train, y_test = Y.iloc[train_index], Y.iloc[test_index]

            dt.fit(X_train, y_train)

            y_pred = dt.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)
            accuracy_scores.append(accuracy)

        mean_accuracy = sum(accuracy_scores) / len(accuracy_scores)
    return mean_accuracy



if __name__ == '__main__':
    # get dataframe path, population number and generation number from command-line argument
    # dataframePath, n_pop, n_gen = getArguments()
    n_pop=50
    n_gen=30
    # read dataframe from csv
    df = pd.read_csv('../../data/iono/ionosphere.csv')
    # encode labels column to numbers
    y = df['target']
    feature_columns = [ col for col in df.columns if col not in ['id','target','f2','f3']]
    for col in feature_columns:
        df[col] = pd.qcut(df[col], q=2, labels=False, duplicates='drop')
    # get accuracy with all features
    feature_columns = [ col for col in df.columns if col not in ['id','target']]
    X = df[feature_columns]
    individual = [1 for i in range(len(X.columns))]
    getFitness(df,'target',individual)
    print("Accuracy with all features: \t" +
          str(getFitness(df,'target',individual)) + "\n")

    # apply genetic algorithm
    hof = geneticAlgorithm(df,'target',n_pop, n_gen)

Accuracy with all features: 	(999,)



C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
C:\ProgramData\Anaconda3\envs\py3.7\lib\site-packages\deap\creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


gen	nevals	avg    	min     
0  	50    	459.877	0.521813
1  	29    	180.324	0.521813
2  	30    	200.273	0.516663
3  	30    	80.4484	0.519069
4  	29    	60.4446	0.37701 
5  	35    	60.4236	0.37888 
6  	21    	0.472535	0.25866 
7  	26    	0.437519	0.25866 
8  	30    	0.402892	0.25866 
9  	27    	0.365533	0.25866 
10 	28    	0.353419	0.25866 
11 	29    	0.313941	0.25866 
12 	33    	0.280748	0.25866 
13 	22    	0.263957	0.25866 
14 	30    	0.281331	0.25866 
15 	31    	0.293264	0.25866 
16 	37    	20.2675 	0.25866 
17 	22    	0.278378	0.25866 
18 	31    	0.287622	0.25866 
19 	29    	20.2564 	0.25866 
20 	33    	0.271854	0.25866 
21 	34    	0.282071	0.25866 
22 	21    	0.281458	0.25866 
23 	33    	0.283533	0.25866 
24 	29    	0.268536	0.25866 
25 	25    	0.291025	0.25866 
26 	30    	0.274106	0.25866 
27 	30    	0.291089	0.25866 
28 	32    	0.286728	0.25866 
29 	29    	0.289549	0.25866 
30 	21    	0.268931	0.25866 
Top 1 solution: [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,

In [65]:
accuracy_KNN,individual = bestIndividual_KNN(hof,df)
print("-----------KNN---------------")
print(accuracy_KNN)

-----------KNN---------------
0.8634126984126984


In [66]:
print('------------特征-----------------')
print(individual)
print(len(individual))

------------特征-----------------
['f3', 'f4', 'f19', 'f22', 'f23', 'f25', 'f31']
7


In [67]:
accuracy_SVM,individual = bestIndividual_SVM(hof,df)
print("-----------SVM---------------")
print(accuracy_SVM)

-----------SVM---------------
0.8888888888888887


In [68]:
print("-----------DT---------------")
accuracy_DT = bestIndividual_DT(hof,df)
print(accuracy_DT)

-----------DT---------------
0.8463492063492062


In [69]:
#seed 42 n_pop=50 n_gen=25 lamda=1
#['f15', 'f18', 'f21', 'f22', 'f25', 'f26', 'f33'] KNN=0.8775 SVM=0.9229 DT=0.869

#seed 42 n_pop=50 n_gen=25 lamda=0.9
#['f15', 'f18', 'f21', 'f22', 'f25', 'f26', 'f33'] KNN=0.8775 SVM=0.9229 DT=0.869

#seed 42 n_pop=50 n_gen=25 lamda=0.8
#['f9', 'f12', 'f18', 'f21', 'f22', 'f24', 'f25', 'f26', 'f33'] KNN=0.8376 SVM=0.9057 DT=0.8889

#seed 42 n_pop=50 n_gen=25 lamda=0.7
#['f9', 'f12', 'f18', 'f21', 'f22', 'f24', 'f25', 'f26', 'f33'] KNN=0.8376 SVM=0.9057 DT=0.8889

#seed 42 n_pop=50 n_gen=25 lamda=0.6
#['f9', 'f12', 'f18', 'f21', 'f22', 'f24', 'f25', 'f26', 'f33'] KNN=0.8376 SVM=0.9057 DT=0.8889

#seed 42 n_pop=50 n_gen=25 lamda=0.5
#['f2', 'f9', 'f14', 'f18', 'f21', 'f25', 'f26', 'f28', 'f33'] KNN=0.8719 SVM=0.9143 DT=0.8775

#seed 42 n_pop=50 n_gen=25 lamda=0.4
#['f2', 'f9', 'f12', 'f14', 'f18', 'f20', 'f21', 'f22', 'f24', 'f25', 'f26', 'f28', 'f33'] KNN=0.8548 SVM=0.9058 DT=0.8976

#seed 42 n_pop=50 n_gen=25 lamda=0.3
#['f2', 'f9', 'f12', 'f14', 'f18', 'f20', 'f21', 'f22', 'f24', 'f25', 'f26', 'f28', 'f33'] KNN=0.8548 SVM=0.9058 DT=0.8976

#seed 42 n_pop=50 n_gen=25 lamda=0.2
#['f2', 'f9', 'f12', 'f14', 'f18', 'f20', 'f21', 'f22', 'f25', 'f26', 'f28', 'f33'] KNN=0.8606 SVM=0.9087 DT=0.8749

#seed 42 n_pop=50 n_gen=25 lamda=0.1
#['f2', 'f9', 'f12', 'f14', 'f18', 'f20', 'f21', 'f22', 'f25', 'f26', 'f28', 'f33'] KNN=0.8606 SVM=0.9087 DT=0.8749

#seed 42 n_pop=50 n_gen=25 lamda=0
#['f2', 'f9', 'f14', 'f18', 'f20', 'f21', 'f22', 'f25', 'f28', 'f32', 'f33'] KNN=0.8633 SVM=0.9287 DT=0.8663

In [70]:
def FN(condition_attrs, data, decision_attr):
    # 计算D的信息熵
    total_records = len(data)
    decision_counts = data[decision_attr].value_counts()
    decision_prob = decision_counts / total_records
    entropy_D = -(decision_prob * np.log2(decision_prob)).sum()

    equivalence_classes = calculate_equivalence_classes(data, condition_attrs)

    equivalence_classes_dec = calculate_equivalence_classes(data, decision_attr)

    rule_confidences = []
    entropy_DB = 0
    total_rule=0

    for equivalence_class in equivalence_classes.values():
        class_size = len(equivalence_class)
        class_prob = class_size / total_records
        for key_decision in equivalence_classes_dec.keys():
            x=set(equivalence_class).intersection(equivalence_classes_dec.get(key_decision))
            if len(x)>total_records*0.01:
                y = 1-len(x) / class_size
                if y<0.3:
                    rule_confidences.append(y)
            if len(x)>0:
                decision_prob = len(x) / class_size
                entropy_DB -= class_prob * decision_prob * math.log(decision_prob, 2)
    rule_confidences = np.array(rule_confidences)
    count = len(rule_confidences)

    print(entropy_D)
    print(entropy_DB)
    print(rule_confidences)
    print(count)

    avg_confidence = np.sum(rule_confidences) / count +0*(entropy_D-entropy_DB)

    return avg_confidence

In [71]:
# df = pd.read_csv('../data/iono/ionosphere.csv')
# feature_columns = [ col for col in df.columns if col not in ['id','target','f2','f3']]
# for col in feature_columns:
#     df[col] = pd.qcut(df[col], q=2, labels=False, duplicates='drop')
#

In [72]:
# feature_columns=['f15', 'f18', 'f21', 'f22', 'f25', 'f26', 'f33']
# # feature_columns01=['f2', 'f9', 'f12', 'f14', 'f18', 'f20', 'f21', 'f22', 'f25', 'f26', 'f28', 'f33']
# print(FN(feature_columns,df,'target'))